### License
This codes of functions was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 13, 2023

In [1]:

import os, json, time
from pyproj import Transformer, Geod
# from wildfire.Reader import Reader as WFReader

In [2]:
import pandas as pd

with open('USGS_Wildland_Fire_Combined_Dataset.json') as json_data:
    data = json.load(json_data)
    df_features = pd.DataFrame(data['features'])

In [3]:
df_features.shape

(135061, 2)

After normalizing the columns in the original dataset to 2 DataFrames, we combine the DataFrames of attributes and geometry of fires into one large DataFrame

Further, we extract fires happened between 1963 and 2023.

In [4]:
df1 = pd.json_normalize(df_features['attributes'])
df2 = pd.json_normalize(df_features['geometry'])
df = pd.concat([df1, df2.reindex(df1.index)], axis=1)
df = df[df['Fire_Year']>=1963][df['Fire_Year']<=2023].reset_index(drop=True)
df.shape

/var/folders/by/rkypwrwj5mg3f381qttxh0s400vssn/T/ipykernel_1367/2926258376.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['Fire_Year']>=1963][df['Fire_Year']<=2023].reset_index(drop=True)


(117578, 32)

In [5]:
# the lat and lon of the city

CITY_LOCATIONS = {
    'Alamogordo' :     {'city'   : 'Alamogordo',
                       'latlon' : [32.8995325, -105.96026499999999] }}

In [12]:
df.isna().sum()

OBJECTID                             0
USGS_Assigned_ID                     0
Assigned_Fire_Type                   0
Fire_Year                            0
Fire_Polygon_Tier                    0
Fire_Attribute_Tiers                 0
GIS_Acres                            0
GIS_Hectares                         0
Source_Datasets                      0
Listed_Fire_Types                    0
Listed_Fire_Names                    0
Listed_Fire_Codes                    0
Listed_Fire_IDs                    824
Listed_Fire_IRWIN_IDs            36413
Listed_Fire_Dates                  300
Listed_Fire_Causes                   0
Listed_Fire_Cause_Class              0
Listed_Rx_Reported_Acres         92573
Listed_Map_Digitize_Methods      13307
Listed_Notes                     35534
Processing_Notes                 32921
Wildfire_Notice                      0
Prescribed_Burn_Notice               0
Wildfire_and_Rx_Flag            115734
Overlap_Within_1_or_2_Flag      101353
Circleness_Scale         

In [14]:
# remove the rings that are of null value

null_ring = df[df['rings'].isna()].index
null_ring

Index([ 92237,  92856,  93271,  94063,  94408,  94529,  95042,  95047,  96043,
        96297,  96370,  96398,  96437,  96941,  96954,  98261,  98606,  98867,
        99718, 102214, 102249, 102383, 102614, 102844, 103063, 103310, 103375,
       103642, 104896, 105163, 106393, 107167, 107678, 108377, 110124],
      dtype='int64')

In [15]:
df = df.drop(null_ring, axis=0)
df.shape

(117543, 32)

In [16]:
df = df.reset_index(drop=True)

In [17]:
# extract the largest ring from rings to do further calculations

f = lambda x : x[0]
biggest_ring = []
float_ring = []
for ring in df['rings']:
    # try:
    biggest_ring.append(f(ring))
    # except:
    #     biggest_ring.append(ring)
    #     float_ring.append(ring)
len(biggest_ring[0])

1133

In [18]:
df['biggest_ring'] = pd.Series(biggest_ring)

In [19]:
#
#    First create a geodesic model that will be used for the calculations. There are a number of
#    different models of the earth. The WSG84 is one that is commonly used and relatively up-to-date
#
#geodcalc = Geod(ellps='clrk66')       # Use Clarke 1866 ellipsoid representation of the earth
geodcalc = Geod(ellps='WGS84')         # Use WGS84 ellipsoid representation of the earth

#    Two constants for accessing the 'latlon' array in our CITY_LOCATIONS constant dict
LAT = 0
LON = 1
#    Get a city from our CITY_LOCATIONS constant as our starting position
start_at = CITY_LOCATIONS["Alamogordo"]

#    Loop through all of the cities to calculate the distance from the starting position
for city_key in CITY_LOCATIONS.keys():
    #    City destination
    destination = CITY_LOCATIONS[city_key]
    #    Note that the 'inv()' function wants coordinates in Longitude,Latitude order by default
    #    inv() also allows lat and lon parameters to be vectors/arrays - in which case the results would be vector/arrarys
    distance = geodcalc.inv(start_at['latlon'][LON],start_at['latlon'][LAT],destination['latlon'][LON],destination['latlon'][LAT])
    #    The 'distance' result variable is a tuple/list with the first two items reflecting forward/backward azimuths
    #    and the third item representing the distance in meters. 
    d_meters = distance[2]
    d_miles = d_meters * 0.00062137 # constant to convert meters to miles
    #    BTW, this isn't actually a 'straight' line because the whole reason for using pyproj is to calculate
    #    these distance measures over the surfase of a sphere/ellipsoid. We set up which ellipsoid to use when we
    #    defined the 'geodcalc' object near the top of this cell
    print(f"Straight line distance from {start_at['city']} to {destination['city']} is {d_meters} meters or {d_miles:5.2f} miles")



Straight line distance from Alamogordo to Alamogordo is 0.0 meters or  0.00 miles


In [20]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

In [21]:
#
#   Convert one ring from the default to EPSG
#
#   There are two options here - depending upon whether you loaded data useing GeoJSON or the wildfire.Reader
#
#ring_in_epsg4326 = convert_ring_to_epsg4326(gj_bigest_ring)
#
ring_in_epsg4326 = convert_ring_to_epsg4326(df['biggest_ring'][0])
#
print(f"Ring consists of {len(ring_in_epsg4326)} points.")
#
#    If you want to print them out you can see what they look like converted.
#print(ring_in_epsg4326)
#for point in ring_in_epsg4326:
#    print(f"{point[0]},{point[1]}")


Ring consists of 1133 points.


In [22]:
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average



In [28]:
def average_distance_from_place_to_fire_perimeter2(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    len_of_ring = len(ring)
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    s = 0
    for i in range(len_of_ring-1):
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],ring[i][1],ring[i][0])
        s += d[2]*0.00062137
    average = s/(len_of_ring-1)
    return average

We are using the average distance between city to fire to determine the distance because we believe there will hardly be many immediate impacts on the city otherwise. To make things easier for Part II and to better investigate impacts, we choose average distance.

In [29]:
from tqdm import tqdm
geodcalc = Geod(ellps='WGS84')
#    Get a city from our CITY_LOCATIONS constant as our starting position
place = CITY_LOCATIONS["Alamogordo"]
latlon = place['latlon']
dist = []
for i in tqdm(range(df.shape[0])):
    ring_data = df.iloc[i]['biggest_ring']
    distance = average_distance_from_place_to_fire_perimeter2(latlon,ring_data)
    dist.append(distance)
df['avg_distance (miles)'] = pd.Series(dist)
df


100%|██████████| 117543/117543 [32:56<00:00, 59.46it/s]


,OBJECTID,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Overlap_Within_1_or_2_Flag,Circleness_Scale,Circle_Flag,Exclude_From_Summary_Rasters,Shape_Length,Shape_Area,rings,curveRings,biggest_ring,avg_distance (miles)
0,14299,14299,Wildfire,1963,1,"1 (1), 3 (3)",40992.458271,16589.059302,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,None,0.385355,NaN,No,73550.428118,1.658906e+08,"[[[-1538222.6155999992, 664547.1687000003], [-...",NaN,"[[-1538222.6155999992, 664547.1687000003], [-1...",972.589109
1,14300,14300,Wildfire,1963,1,"1 (1), 3 (3)",25757.090203,10423.524591,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (2)",...,None,0.364815,NaN,No,59920.576713,1.042352e+08,"[[[-1561373.1005000006, 710862.5734999999], [-...",NaN,"[[-1561373.1005000006, 710862.5734999999], [-1...",999.009827
2,14301,14301,Wildfire,1963,1,"1 (5), 3 (15), 5 (1)",45527.210986,18424.208617,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (6), Likely Wildfire (15)",...,None,0.320927,NaN,No,84936.827810,1.842421e+08,"[[[-1516651.6959000006, 689701.9166000001], [-...",NaN,"[[-1516651.6959000006, 689701.9166000001], [-1...",972.496058
3,14302,14302,Wildfire,1963,1,"1 (1), 3 (3), 5 (1)",10395.010334,4206.711433,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (3)",...,None,0.428936,NaN,No,35105.903602,4.206711e+07,"[[[-1495760.2166000009, 539132.1772000007], [-...",NaN,"[[-1495760.2166000009, 539132.1772000007], [-1...",888.681551
4,14303,14303,Wildfire,1963,1,"1 (1), 3 (3)",9983.605738,4040.221900,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (1), Likely Wildfire (3)",...,None,0.703178,NaN,No,26870.456126,4.040222e+07,"[[[-1520641.9629999995, 654341.4522999991], [-...",NaN,"[[-1520641.9629999995, 654341.4522999991], [-1...",956.661335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117538,135057,135057,Prescribed Fire,2020,8,8 (3),16.412148,6.641761,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (3),...,"Caution, this Prescribed Fire in 2020 overlaps...",0.177425,NaN,No,2168.900740,6.641761e+04,"[[[-2008802.4960999992, 729335.5622000005], [-...",NaN,"[[-2008802.4960999992, 729335.5622000005], [-2...",1181.649834
117539,135058,135058,Prescribed Fire,2020,8,8 (1),7.050837,2.853373,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,"Caution, this Prescribed Fire in 2020 overlaps...",0.374368,NaN,No,978.666221,2.853373e+04,"[[[-1648510.3187000006, 666410.7272999994], [-...",NaN,"[[-1648510.3187000006, 666410.7272999994], [-1...",1007.002978
117540,135059,135059,Prescribed Fire,2020,8,8 (4),9.342668,3.780843,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (4),...,"Caution, this Prescribed Fire in 2020 overlaps...",0.123888,NaN,No,1958.326660,3.780843e+04,"[[[-1649244.5655000005, 664993.7576000001], [-...",NaN,"[[-1649244.5655000005, 664993.7576000001], [-1...",1006.448344
117541,135060,135060,Prescribed Fire,2020,8,8 (1),0.996962,0.403456,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,None,0.993809,1.0,No,225.866452,4.034562e+03,"[[[-1017808.4253000002, 140344.1116000004], [-...",NaN,"[[-1017808.4253000002, 140344.1116000004], [-1...",554.443228


In [30]:
df.to_csv('full_dataset.csv', index=False)

Here we only take in the fires whose centers are within 1250 miles from the city

In [31]:
df = df[df['avg_distance (miles)'] <= 1250]
df.shape

(95064, 34)

In [32]:
df.to_csv('city.csv', index=False)